# Final notes on Version v02

In [1]:
# imports
import pdb
from specdb.specdb import IgmSpec

from astropy import units as u
from astropy.coordinates import match_coordinates_sky, SkyCoord

#from xastropy.obs import radec as xra

## Load

In [2]:
igmsp = IgmSpec()

Using /raid/IGMSPEC_DB/IGMspec_DB_v02.hdf5 for the catalog file
Using /raid/IGMSPEC_DB/IGMspec_DB_v02.hdf5 for the DB file
Available surveys: [u'BOSS_DR12', u'HSTQSO', u'SDSS_DR7', u'KODIAQ_DR1', u'HD-LLS_DR1', u'2QZ', u'ESI_DLA', u'HDLA100', u'GGG', u'COS-Halos', u'HST_z2', u'COS-Dwarfs', u'XQ-100']
Database is igmspec
Created on 2016-Oct-09


## Pairs

In [3]:
pair_sep=10*u.arcsec

In [4]:
def chk_for_pairs(maindb, pair_sep=10*u.arcsec):
    """ Generate new IGM_IDs for an input DB

    Parameters
    ----------
    maindb : Table

    Return
    ------
    result : bool
      * True = pass
      * False = fail
    """
    c_main = SkyCoord(ra=maindb['RA'], dec=maindb['DEC'], unit='deg')
    # Find candidate dups
    idx, d2d, d3d = match_coordinates_sky(c_main, c_main, nthneighbor=2)
    cand_pairs = np.where(d2d < pair_sep)[0]
    # Finish
    print("There are {:d} potential pairs with separation theta<{:g}".format(len(cand_pairs)/2,pair_sep))
    return cand_pairs

In [5]:
cpairs = chk_for_pairs(igmsp.qcat.cat)

There are 185 potential pairs with separation theta<10 arcsec


### Print a list

In [9]:
c_main = SkyCoord(ra=igmsp.qcat.cat['RA'], dec=igmsp.qcat.cat['DEC'], unit='deg')

In [12]:
flg_cp = np.array([False]*len(igmsp.qcat.cat))
for qq, cpair in enumerate(cpairs):
    # Skip those already done
    if flg_cp[cpair]:
        continue
    # Find the matches
    sep = c_main[cpair].separation(c_main)
    pairs = sep < pair_sep
    flg_cp[pairs] = True
    # Skip pairs with very different zem
    if np.sum(pairs) == 2:
        zem = igmsp.qcat.cat['zem'][pairs]
        if np.abs(zem[0]-zem[1]) > 0.1:
            continue
        # Both BOSS?
        if (igmsp.qcat.cat['flag_survey'][pairs][0] == 1.) & (
            igmsp.qcat.cat['flag_survey'][pairs][1] == 1.):
            continue
    # Skip table?
    if np.min(np.abs(igmsp.qcat.cat['RA'][pairs][0]-stbl['RA'])) < 1e-4:
        continue
    # XQ-100?
    if igmsp.qcat.cat['flag_survey'][pairs][1] == 64.:
        print("Skipping XQ-100")
        pdb.set_trace()
        continue
    # Print
    print('qq = {:d}'.format(qq))
    print(igmsp.qcat.cat[['RA','DEC','IGM_ID','zem','flag_survey']][pairs])
    print(sep.to('arcsec')[pairs])
    pdb.set_trace()

qq = 30
   RA      DEC    IGM_ID  zem   flag_survey
-------- -------- ------ ------ -----------
 32.4359   5.2871  32570  4.141       133.0
 32.4355   5.2881 382423  4.170         8.0
[u'0arcsec' u'3.9306arcsec']
> <ipython-input-12-0f8c1c474fbb>(2)<module>()
-> for qq, cpair in enumerate(cpairs):
(Pdb) c
qq = 31
   RA      DEC    IGM_ID  zem   flag_survey
-------- -------- ------ ------ -----------
 32.4613  -0.0851  32589  2.858        15.0
 32.4629  -0.0869 382424  2.860         8.0
[u'0arcsec' u'8.72939arcsec']
> <ipython-input-12-0f8c1c474fbb>(2)<module>()
-> for qq, cpair in enumerate(cpairs):
(Pdb) c
qq = 203
   RA      DEC    IGM_ID  zem   flag_survey
-------- -------- ------ ------ -----------
216.9947  -1.3601 218288  2.354         1.0
216.9954  -1.3584 218290  2.278       257.0
[u'0arcsec' u'6.2357arcsec']
> <ipython-input-12-0f8c1c474fbb>(2)<module>()
-> for qq, cpair in enumerate(cpairs):
(Pdb) c
qq = 324
   RA      DEC    IGM_ID  zem   flag_survey
-------- -------- ------

In [ ]:
tdF_bad = [[181.7848,  -0.9993],
      ]

In [6]:
skip = [[10.9697,   4.4073],
        [15.3189,   2.0326],
        [23.7452,  24.5140],
        [28.4283,  20.9148],
        [35.1739,   1.1985],
        [40.0218,  -0.6527],
        [41.2998,  -1.2216],
        [41.9780,   0.6380],
        [123.3026,  54.2806],
        [126.6732,  45.7450],
        [131.5022,   7.0747],
        [150.3681,  50.4663],
        [150.3362,  55.8989],  # FOS lens
        [158.2551,  47.2532],
        [164.0395,  55.2669],
        [170.1281,  54.7426],
        [176.7206,  16.7400],
        [188.5052,   6.5367],
        [190.7380,  25.7174],
        [193.7286,   8.7812],
        [196.9841,   4.3710],
        [198.7737,  47.9047],
        [201.3239,  37.6164],
        [211.2581,  44.8000],
        [222.7320,  47.0272],
        [238.3773,  22.5040],
        [243.2571,   8.1350],
        [253.7555,  26.0882],
        [357.0800,   0.9549],
        [116.9959,  43.3015],
        [184.6687,  50.2621],
        [166.63912,   -18.35661],  # FOS lens
        [166.6396, -18.3567], # FOS lens
        [216.9947,  -1.3601],  
        [9.9763, -27.4229],  # 2QZ pair
        [341.6578, -29.4963], # 2QZ pair
        ]

In [7]:
sa = np.array(skip)
stbl = Table()
stbl['RA'] = sa[:,0]
stbl['DEC'] = sa[:,1]
#stbl

### Bad sources

In [42]:
bad = [[21.0166,   0.7421],  # XQ-100
       [23.4187,   4.0163],  # XQ-100
       [28.4156,  -0.1850],  # XQ-100
       [32.4355,   5.2881],  # HD-LLS, HD100
       [32.4629,  -0.0869],  # HD-LLS (Q0207)
       [43.8279,   0.8129],  # XQ-100
       [129.8876,  11.2011],  # HD100
       [131.1012,  12.7635],  # HD100
       [143.4076,  28.7598], # HD100
       [149.4353,  33.1399], # HD100
       [150.3368,  55.8971], # HST-QSO, SDSS, BOSS :: Very strange..
       [155.4868,  30.0254], #HD100
       [164.4842,  45.9290],  #HD100
       [173.7835,  22.4519],  #HD100
       [216.7352,  60.4285],  #HD100
       [338.5367,   0.0006],  #HD100
       [356.6908,  12.7583],  #HD100
       [170.3085,   3.4297],   # Bad COS
       [332.2169, -19.7327],  #HD100
       [9.9763, -26.5771],    # Not in 2QZ?! -- Nope, my bug
       [188.0004,  -2.4021],   # Bad HSTQSO
       [17.5677,  -2.3147 ],   # Bad HSTQSO
      ]

## Nsources

In [6]:
len(igmsp.qcat.cat)

405189

## NSpec

In [5]:
Nspec = 0
for key in igmsp.idb.hdf.keys():
    if key in ['catalog', 'quasars']:
        continue
    #
    Nspec += igmsp.idb.hdf[key]['meta'].size
#
print("Nspec = {:d}".format(Nspec))

Nspec = 433910
